In [ ]:
# Import of important libraries in Python:
import sys, os, time, importlib, glob, pdb
import matplotlib, pylab as pl, numpy as np
from numpy import log10 as log
import h5py, astropy, scipy, scipy.stats
from astropy import units as un, constants as cons
from scipy import interpolate
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits import mplot3d
import statistics as s
import random
from os import listdir
from os.path import isfile, join
import MyTools.AnalysisTools as l
import MyTools.HalosData as HD
import MyTools.Verdict as Verdict
from MyTools.Project3MgIIAnalysisConfigs import ProjectionsOfCoolGasFractionInSightlinesConfig as c
import math
import matplotlib.patches as mpatches
from astropy import units as un, constants as cons
import pylab as pl
import seaborn as sns
import palettable
%matplotlib auto

# FIRE studio libraries
from OldFireStudio.firestudio.studios.gas_studio import GasStudio
from OldFireStudio.firestudio.studios.star_studio import StarStudio
from GitHub import FIRE_files as ff
from GitHub import simulationFiles

# Important constants
X=0.7 # Hydrogen mass fraction
h=0.7 # Hubble parameter in units of 100 km/s Mpc^-1
kpc_3Tocm_3Factor=2.94e64 # kpc^3 to cm^3 multiply factor
ProtonMassInSunMass=8.4e-58 # Proton mass in solar mass

# Print the config file:
# For Stampede
print("For Stampede:")
print("The simulations names in the config file are:",c.ListOfStampedeSimulationsNames)
print("The simulations key names in the config file are:",c.ListOfStampedekeysOfTheSimulationsNames)
print("The snapshot numbers in the config file are:",c.ListOfStampedeSnapshotsNumbers)
print("")

In [ ]:
# Creates the list of the default simulationes from the config file and the relevant paths for them
ListOfStampedeSimulationsNames=c.ListOfStampedeSimulationsNames
ListOfStampedeSimulationsPaths=[]
for i in ListOfStampedeSimulationsNames:
    SnapDirect=c.StampedeSnapDirect
    SnapDirect=SnapDirect+i
    ListOfStampedeSimulationsPaths.append(SnapDirect)
print("The simulations paths for Stampede are: ",ListOfStampedeSimulationsPaths)
print("The snapshots numbers for Stampede are: ",c.ListOfStampedeSnapshotsNumbers)

In [ ]:
# This block will create the full paths directories for the chosen simulations and sanpshot numbers
ListOfSnapshotsNumbersForStampede=c.ListOfStampedeSnapshotsNumbers
ListOfOutputFolderPathsForStampede=[]
for i in ListOfStampedeSimulationsPaths:
    temp=i+"/output"
    ListOfOutputFolderPathsForStampede.append(temp)
ListOfFullPathsForStampede=[]
for i in ListOfOutputFolderPathsForStampede:
    OutputFiles=os.listdir(i)
    for j in ListOfSnapshotsNumbersForStampede:
        option1="snapshot_"+j+".hdf5"
        option2="snapdir_"+j
        if(option1 in OutputFiles):
            FullPath=i+"/"+option1
        if(option2 in OutputFiles):
            FullPath=i+"/"+option2
        ListOfFullPathsForStampede.append(FullPath) 
print("The full paths for Stampede simulations and snapshots are:")
print(ListOfFullPathsForStampede)

In [ ]:
# Useful functions
def ionFractions(z,Ts,nHs,tablefn,element='Mg',ionizationLevel=1):
    # Returns the MgII fraction
    F=h5py.File(tablefn,'r')
    logTs_hm2012=np.array(F[element].attrs['Temperature'])
    lognHs_hm2012=np.array(F[element].attrs['Parameter1'])
    zs_hm2012=np.array(F[element].attrs['Parameter2'])
    ind_z=np.searchsorted(zs_hm2012,z)
    log_f_tab=np.array(F[element])[ionizationLevel,:,ind_z,:]
    func=interpolate.RectBivariateSpline(lognHs_hm2012,logTs_hm2012,log_f_tab)
    res=func.ev(log(nHs),log(Ts))
    F.close()
    return (10**res)

# Creates a list of the redshifts in m12i_res7100 data (the files Jonathan sent)
ListOfFiles=os.listdir("/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Project2-OutflowsCGM/m12i_res7100Data/")
ListOfRedshiftsTheo=[]
for i in ListOfFiles:
    RedshiftTheo=i.replace("._profiler_m12i_md_7100_z","")
    RedshiftTheo=RedshiftTheo.replace("profiler_m12i_md_7100_z","")
    RedshiftTheo=RedshiftTheo.replace(".npz","")
    RedshiftTheo=float(RedshiftTheo)
    ListOfRedshiftsTheo.append("{:.4f}".format(RedshiftTheo))
    
def FindClosest(lst,K):
    # Finds the closest element in an array for a given value
    return lst[min(range(len(lst)),key=lambda i: abs(float(lst[i])-K))]
    
def Cartesian_To_Spherical(x,y,z):
    # Calculates spherical theta and phi for a given caratezian coordinates [x,y,z]
    theta=math.atan2(math.sqrt(x**2+y**2),z)
    phi=math.atan2(y,x) if x >= 0 else math.atan2(y,x) + math.pi
    return theta, phi

In [ ]:
# This block will create MgII projections plots for different velocities in range [-400,400]km/s for different snapshots
# Output: will save 2 files: .png image of the plot and .hdf5 file of the projection
plt.clf()
ListOfSnapshotsParts=[]
forloopcount=0
for SnapDirect in ListOfFullPathsForStampede:
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append Only hdf5 Files To The Snapshots Parts List
    print("The list of the snapshot parts is:",ListOfSnapshotsParts)
    print("")
    SimulationKey=c.ListOfStampedekeysOfTheSimulationsNames[forloopcount]
    Snap1=l.Snapshot(ListOfSnapshotsParts[0],[0,0,0],[0,0,0])# SnapName, Not Real Center Coordinates, Not Real Center Velocity
    z=Snap1.Redshift()
    print("The redshift of the given halo is:",z)
    Mvir=HD.HaloMvir(SimulationKey,z)
    Rvir=HD.HaloRvir(SimulationKey,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationKey,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationKey,z)
    afactor=HD.HaloaFactor(SimulationKey,z)

    # This part intended to merge all the data from the different snapshot parts (if there are more than 1 part)
    TotalNumberOfParticles=0
    GasCoordinates=[]
    StarsCoordinates=[]
    GasTemperatures=[]
    GasTemperatures0or1=[]
    GasRadiuses=[]
    StarsRadiuses=[]
    GasDensity=[]
    GasVolume=[]
    NeutralH=[]
    GasMasses=[]
    StarsMasses=[]
    BoxesSizes=[]
    GasSmoothingLengths=[]
    GasVelocities=[]
    StarsVelocities=[]
    Metallicity=[]
    nHs=[]
    JVector=[0,0,0]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, Real Center Coordinates, Real Center Velocity

        NumberOfParticlesOfOneSnapshot=Snap1.NumberOfParticles()# 0-Gas,1-High Resolution Dark Matter,2-Dummy Particles,3-Dummy Particles,4-Stars,5-Black Holes
        TotalNumberOfParticles=TotalNumberOfParticles+NumberOfParticlesOfOneSnapshot[0]

        GasMassesOfOneSnapshot=Snap1.MassesInSunMasses(0)# The masses of the gas particles in [M_sun]
        GasMasses.extend(GasMassesOfOneSnapshot)
        
        StarsMassesOfOneSnapshot=Snap1.MassesInSunMasses(4)# The masses of the stars particles in [M_sun]
        StarsMasses.extend(StarsMassesOfOneSnapshot)

        GasCoordinatesOfOneSnapShot=Snap1.CoordinatesInkpc(0)# The coordinates of the gas particles in [kpc]
        GasCoordinates.extend(GasCoordinatesOfOneSnapShot)
        
        StarsCoordinatesOfOneSnapShot=Snap1.CoordinatesInkpc(4)# The coordinates of the stars particles in [kpc]
        StarsCoordinates.extend(StarsCoordinatesOfOneSnapShot)

        GasTemperaturesOfOneSnapShot=Snap1.TemperaturesInK()# The temperature of the gas particles in [K]
        GasTemperatures.extend(GasTemperatureOfOneSnapShot)

        GasRadiusesOfOneSnapShot=Snap1.RadiusesInkpc(0)# The r Of the gas particles in [kpc]
        GasRadiuses.extend(GasRadiusesOfOneSnapShot)

        StarsRadiusesOfOneSnapShotStarsParticles=Snap1.RadiusesInkpc(4)# The r of the stars particles in [kpc]
        StarsRadiuses.extend(StarsRadiusesOfOneSnapShotStarsParticles)
        
        GasDensityOfOneSnapShot=Snap1.DensityIn_gr_cm_3(0)# The density of the gas particles in [gr/cm^3]
        GasDensity.extend(GasDensityOfOneSnapShot)
        
        GasVolumeOfOneSnapShot=Snap1.VolumeInkpc_3()# The volume of the gas particles nn [kpc^3]
        GasVolume.extend(GasVolumeOfOneSnapShot)

        NeutralHOfOneSnapshot=Snap1.NeutralHydrogenAbundance(0)# The neutral hydrogen fraction of the gas particles
        NeutralH.extend(NeutralHOfOneSnapshot)
        
        GasSmoothingLengthOfOneSnapshot=Snap1.SmoothingLength(0)# The smoothingLength of the gas particles
        GasSmoothingLengths.extend(GasSmoothingLengthOfOneSnapshot)
        
        GasVelocitiesOfOneSnapshot=Snap1.VelocitiesIn_km_s(0) # The velocities of the gas particles in [km/s]
        GasVelocities.extend(GasVelocitiesOfOneSnapshot)
        
        StarsVelocitiesOfOneSnapshot=Snap1.VelocitiesIn_km_s(4) # The velocities of the stars particles in [km/s]
        StarsVelocities.extend(StarsVelocitiesOfOneSnapshot)
        
        MetallicityOfOneSnapshot=Snap1.Metallicity(0) # The metallicity of the gas particles 
        Metallicity.extend(MetallicityOfOneSnapshot)
        
        nHsOfOneSnapshot=Snap1.nHs(0) # The particles density of the gas particles in [cm^-3]
        nHs.extend(nHsOfOneSnapshot)
        
        JVectorOfOneSnapshot=Snap1.JVectores(4) # The stars J vectores in [M_sun*kpc^2/s]
        JVectorOfOneSnapshot=np.sum(JVectorOfOneSnapshot,axis=0)
        JVector=JVector+JVectorOfOneSnapshot
    
    ConversionTables='/home1/08289/tg875885/MgIIAnalysis/hm2012_hr.h5'
    MgII_Mg_Fractions=ionFractions(z,GasTemperature,nHs,ConversionTables)
    
    GasCoordinates=np.array(GasCoordinates)
    StarsCoordinates=np.array(StarsCoordinates)
    GasTemperatures=np.array(GasTemperature)
    GasRadiuses=np.array(GasRadiuses)
    StarsRadiuses=np.array(StarsRadiuses)
    GasDensity=np.array(GasDensity)
    GasVolume=np.array(GasVolume)
    NeutralH=np.array(NeutralH)
    GasMasses=np.array(GasMasses)
    StarsMasses=np.array(StarsMasses)
    GasSmoothingLengths=np.array(GasSmoothingLengths)
    GasVelocities=np.array(GasVelocities)
    StarsVelocities=np.array(StarsVelocities)
    Metallicity=np.array(Metallicity)
    nHs=np.array(nHs)
    MgII_Mg_Fractions=np.array(MgII_Mg_Fractions)
    
    for temp in GasTemperatures:
        if(temp<30000):
            GasTemperatures0or1.append(1)
        else:
            GasTemperatures0or1.append(0)
    GasTemperatures0or1=np.array(GasTemperatures0or1)
    
    Vzs=[]
    for t in GasVelocities:
        Vzs.append(t[2])
    Vzs=np.array(Vzs)
    
    MgFractions=[]
    for t in Metallicity:
        MgFractions.append(t[6])
    MgFractions=np.array(MgFractions)
    
    print("The total number of gas particles is:",TotalNumberOfParticles)
    print("r min is:",GasRadiuses.min())
    print("r max is:",GasRadiuses.max())
          
    # Save the data of the particles in each bin
    SnapDirect=SnapDirect.replace("/","-")
    SnapDirect=SnapDirect.replace(".hdf5","")
    
print("DONE")
####################################################################################################################################################

In [ ]:
# Find the angular momentum vector of the stars within 0.2Rvir
count=0
JVectorOfStarsWithin0_2Rvir=np.array([0,0,0])
for i in range(0,len(StarsRadiuses)):
    if(StarsRadiuses[i]<0.2*(Rvir)):
        count+=1
        J_vec=StarsMasses[i]*np.cross(StarsCoordinates[i],StarsVelocities[i])
        JVectorOfStarsWithin0_2Rvir=JVectorOfStarsWithin0_2Rvir+J_vec
print(JVectorOfStarsWithin0_2Rvir)

# Find the angular momentum vector of the gas within 0.2Rvir
count=0
JVectorOfGasWithin0_2Rvir=np.array([0,0,0])
for i in range(0,len(GasRadiuses)):
    if(GasRadiuses[i]<0.2*(Rvir)):
        count+=1
        J_vec=GasMasses[i]*np.cross(GasCoordinates[i],GasVelocities[i])
        JVectorOfGasWithin0_2Rvir=JVectorOfGasWithin0_2Rvir+J_vec
print(JVectorOfGasWithin0_2Rvir)

In [ ]:
UnitJvector=JVectorOfStarsWithin0_2Rvir/np.linalg.norm(JVectorOfStarsWithin0_2Rvir)
print(UnitJvector)
new_orientation=(np.array([0,0,1]),np.array([0,1,0]))[True]
rotvec=np.cross(UnitJvector,new_orientation)
sin_theta=np.linalg.norm(rotvec)
rotvec_normed=-rotvec/sin_theta*np.arcsin(sin_theta)
rot1=Rotation.from_rotvec(rotvec_normed)
RotatedCoordinates=rot1.apply(GasCoordinates)

In [ ]:
# Create a projection for each velocity bin of T<30000K

# Load data
SimulationName=c.ListOfStampedekeysOfTheSimulationsNames[forloopcount]
SimulationData=[] # SnapNumber, Redshift, Mvir, Rvir, Xs, Ys, MgIImap
SnapDirect="-scratch-projects-xsede-GalaxiesOnFIRE-metal_diffusion-m12b_r7100-output-snapdir_"
SnapNumber=513
z=HD.Redshift("m12b",SnapNumber)
Mvir=HD.HaloMvir("m12b",z)
Rvir=HD.HaloRvir("m12b",z)

snapdir='/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/output'
iSnapshot=SnapNumber
r_max=50
z_width=30
filedir='/home1/08289/tg875885/radial_to_rotating_flows/Aharon/'
SnapDirect=SnapDirect.replace("/","-")
SnapDirect=SnapDirect.replace(".hdf5","")
projection_output_filename=SnapDirect+str(SnapNumber)

# Create input dictionary for FIRE studio from snapshot and snapdict
studiodic = {}
studiodic['Coordinates']    = RotatedCoordinates
studiodic['Masses']         = (GasVolume*GasTemperatures0or1*(ProtonMassInSunMass)*(kpc_3Tocm_3Factor))/X
studiodic['BoxSize']        = Snap1.BoxSize()
studiodic['SmoothingLength']= GasSmoothingLengths

# Create the projection 
mystudio=GasStudio(
                snapdir, 
                snapnum=int(iSnapshot),
                snapdict=studiodic,
                datadir=filedir,
                frame_half_width=r_max,
                frame_depth=z_width,
                quantity_name='Masses',
                take_log_of_quantity=False, 
                galaxy_extractor=False,
                pixels=1200,
                single_image='Density',
                overwrite=True,
                savefig=False,      
                use_hsml=True,
                intermediate_file_name=projection_output_filename,
                )

fig,ax=plt.subplots()
MgIImap,_=mystudio.projectImage([])
MgIImap+=log((X*un.Msun*un.pc**-2/(cons.m_p)/1e10*h).to('cm**-2').value) # Units fix
MgIImap=MgIImap.T
MgIImap=10**MgIImap
MgIImap=MgIImap*un.cm.to('kpc')
Xs=np.linspace(-r_max,r_max,MgIImap.shape[0])
Ys=np.linspace(-r_max,r_max,MgIImap.shape[1])

# This part prepares the graph
minNorm=0
maxNorm=30/60
plt.rcParams['mathtext.fontset']='dejavuserif'
plt.pcolormesh(Xs,Ys,MgIImap/60,cmap='inferno',norm=pl.Normalize(minNorm,maxNorm))
cbar=pl.colorbar()
plt.xlabel("X [kpc]",fontsize=12,fontname="serif")
plt.ylabel("Y [kpc]",fontsize=12,fontname="serif")
#title="z= "+str(round(z,2))+"   ;   M= "+str(Mvir)+r" $M_{☉}$"+"\n"+"Bin: "+str(Bins[t])+" "+r"$\frac{km}{s}$"
#plt.title(title)
#a_circle=plt.Circle((0,0),Rvir,fill=False,color='black') # Draw white circle in Rvir of the halo
#ax.add_patch(a_circle) # Adds the circle to the plot
plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/OutpotsMgII-ProjectionsOfCoolGasFractionInSightlines/Projections/'+SimulationName+'/'+SnapDirect+str(SnapNumber)+'_('+str(minNorm)+','+str(maxNorm)+')'+'.png')
SimulationData.append(SnapNumber)
SimulationData.append(z)
SimulationData.append(Mvir)
SimulationData.append(Rvir)
SimulationData.append(Xs)
SimulationData.append(Ys)
SimulationData.append(MgIImap)
SimulationData=np.array(SimulationData)
np.savez_compressed('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/OutpotsMgII-ProjectionsOfCoolGasFractionInSightlines/Data/'+SimulationName+'/'+SnapDirect+str(SnapNumber)+'.npz',SimulationData,dtype=float)

plt.show()
plt.clf()
plt.cla()
plt.close()

print("DONE")

In [ ]:
from IPython.lib.display import Audio

framerate=4410
play_time_seconds=3

t=np.linspace(0,play_time_seconds,framerate*play_time_seconds)
audio_data=np.sin(2*np.pi*300*t)+np.sin(2*np.pi*240*t)
Audio(audio_data,rate=framerate,autoplay=True)